# Interactive Gaussian Beam Mode Matching Tool

A user-friendly GUI for designing and optimizing optical systems for Gaussian beam mode matching.

In an optics lab, coupling light efficiently from one optical component into another (for example an from free space into an optical fiber) is extremely important. The process of optimizing this coupling is called mode-matching and it is typically implemented with lenses.

---

## Gaussian Beam Mode Matching: Theory and Implementation

### 1. Gaussian Beam Fundamentals

#### 1.1 The Complex Beam Parameter

The state of a Gaussian beam at any point in space can be completely described by the **complex beam parameter** $q$:

$$q(z) = z - z_0 + i z_R$$

where:
- $z$ is the axial position along the beam
- $z_0$ is the waist location
- $z_R$ is the Rayleigh range

At the beam waist ($z = z_0$), the complex beam parameter simplifies to:

$$q(z_0) = i z_R$$

#### 1.2 Rayleigh Range

The **Rayleigh range** characterizes how quickly a Gaussian beam diverges:

$$z_R = \frac{\pi w_0^2}{\lambda}$$

where:
- $w_0$ is the beam waist radius (minimum spot size)
- $\lambda$ is the wavelength

At a distance of one Rayleigh range from the waist, the beam area doubles.

#### 1.3 Beam Width Evolution

The beam radius at any position $z$ is given by:

$$w(z) = w_0 \sqrt{1 + \left(\frac{z - z_0}{z_R}\right)^2}$$

This produces the characteristic "bowtie" shape of a Gaussian beam, with minimum at the waist.

#### 1.4 Radius of Curvature

The wavefront radius of curvature is:

$$R(z) = (z - z_0) \left[1 + \left(\frac{z_R}{z - z_0}\right)^2\right]$$

At the waist, $R = \infty$ (plane wave). Far from the waist, $R \approx z - z_0$ (spherical wave).

#### 1.5 Gouy Phase

The accumulated phase shift relative to a plane wave:

$$\psi(z) = \arctan\left(\frac{z - z_0}{z_R}\right)$$

This represents an additional $\pi$ phase shift as the beam passes through its waist.

---

### 2. ABCD Matrix Formalism

#### 2.1 Ray Transfer Matrices

Optical systems can be represented by 2×2 ABCD matrices that describe how rays transform:

$$\begin{pmatrix} r' \\ \theta' \end{pmatrix} = \begin{pmatrix} A & B \\ C & D \end{pmatrix} \begin{pmatrix} r \\ \theta \end{pmatrix}$$

where $r$ is the ray height and $\theta$ is the ray angle.

#### 2.2 Common ABCD Matrices

**Free space propagation** by distance $d$:

$$M_{\text{space}} = \begin{pmatrix} 1 & d \\ 0 & 1 \end{pmatrix}$$

**Thin lens** with focal length $f$:

$$M_{\text{lens}} = \begin{pmatrix} 1 & 0 \\ -\frac{1}{f} & 1 \end{pmatrix}$$

#### 2.3 Transformation of Complex Beam Parameter

The power of the ABCD formalism is that the complex beam parameter transforms as:

$$q' = \frac{Aq + B}{Cq + D}$$

**Key insight:** For free space, $q' = q + d$ (simple addition)

For a thin lens, $q' = \frac{q}{1 - q/f}$ (which comes from the transformation with $A=1, B=0, C=-1/f, D=1$)

---

### 3. Extracting Physical Parameters from $q$

#### 3.1 Beam Width from $q$

Given $q$ at a position $z$, the beam width is:

$$w = \sqrt{\frac{\lambda}{\pi \text{Im}(1/q)}}$$

Equivalently:

$$w = \sqrt{\frac{-\lambda}{\pi} \cdot \frac{1}{\text{Im}(1/q)}}$$

#### 3.2 Radius of Curvature from $q$

$$R = \frac{1}{\text{Re}(1/q)}$$

If $\text{Re}(1/q) = 0$, then $R = \infty$ (at the waist).

---

### 4. Beam Propagation Through Optical Systems

#### 4.1 Algorithm

To find the beam at position $z$ in an optical system:

1. **Initialize** at the waist: $q = i z_R$, at position $z_0$
2. For each optical element (in order):
   - **Propagate** in free space to element: $q \rightarrow q + d$
   - **Transform** through element: $q \rightarrow \frac{Aq + B}{Cq + D}$
3. **Propagate** to final position $z$
4. **Extract** physical parameters from $q$

### 5. Mode Matching

#### 5.1 Overlap Integral

The power coupling efficiency between two Gaussian beams at position $z$ is given by the **mode overlap integral**:

$$\eta = \left|\int_{-\infty}^{\infty} u_1^*(x,y) \cdot u_2(x,y) \, dx \, dy\right|^2$$

For Gaussian beams, this has a closed-form solution:

$$\eta = \frac{4}{\left(\frac{w_1}{w_2} + \frac{w_2}{w_1}\right)^2 + \left(\frac{\lambda \pi}{w_1 w_2}\right)^2 \left(\frac{1}{R_1} - \frac{1}{R_2}\right)^2}$$

where:
- $w_1, w_2$ are the beam radii
- $R_1, R_2$ are the radii of curvature
- $\lambda$ is the wavelength

### 5.2 Perfect Mode Match

Maximum coupling ($\eta = 1$) occurs when:
1. Beam sizes match: $w_1 = w_2$
2. Curvatures match: $R_1 = R_2$

This typically means matching both position and size of the waist.

### 5.3 Optimization Strategy

To optimize mode matching:
1. Define target beam (usually a waist at specific location)
2. Parameterize lens positions as variables
3. Propagate actual beam through system
4. Compute overlap at target location
5. Use numerical optimization to maximize overlap

---


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, curve_fit
from dataclasses import dataclass
from typing import List, Tuple, Optional
from copy import deepcopy
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import io
import base64

plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

## Core Beam Calculation Functions

In [7]:
def rayleigh_range(w0, wavelength=1064e-9):
    return np.pi * w0**2 / wavelength

def beam_width(z, w0, z0, wavelength=1064e-9):
    zR = rayleigh_range(w0, wavelength)
    return w0 * np.sqrt(1 + ((z - z0) / zR)**2)

def gouy_phase(z, w0, z0, wavelength=1064e-9, wrap=False):
    zR = rayleigh_range(w0, wavelength)
    phase = np.arctan2((z - z0), zR) * 180.0 / np.pi
    if wrap:
        phase = np.mod(phase + 180, 360) - 180
    return phase

def complex_beam_parameter(z, w0, z0, wavelength=1064e-9):
    zR = rayleigh_range(w0, wavelength)
    return (z - z0) + 1j * zR

def fit_gaussian_beam(z_data, w_data, wavelength=1064e-9):
    def model(z, w0, z0):
        return beam_width(z, w0, z0, wavelength)
    
    min_idx = np.argmin(w_data)
    w0_guess = w_data[min_idx] * 0.8
    z0_guess = z_data[min_idx]
    if min_idx == 0:
        z0_guess = z_data[0] - 0.1
    
    bounds = ([1e-6, -100], [10e-3, 100])
    
    try:
        popt, pcov = curve_fit(
            model, z_data, w_data,
            p0=[w0_guess, z0_guess],
            bounds=bounds,
            maxfev=50000,
            ftol=1e-12,
            xtol=1e-12
        )
        w0_fit, z0_fit = popt
        residuals = w_data - model(z_data, w0_fit, z0_fit)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((w_data - np.mean(w_data))**2)
        r_squared = 1 - (ss_res / ss_tot)
        return w0_fit, z0_fit, r_squared
    except Exception as e:
        return w0_guess, z0_guess, 0.0

## BeamPath Class

In [8]:
@dataclass
class Lens:
    position: float
    focal_length: float
    label: str

class BeamPath:
    def __init__(self, wavelength=1064e-9):
        self.wavelength = wavelength
        self.w0 = None
        self.z0 = None
        self.lenses: List[Lens] = []
        self.target_w0 = None
        self.target_z0 = None
    
    def seed_waist(self, w0, z0):
        self.w0 = w0
        self.z0 = z0
        return self
    
    def add_lens(self, focal_length, position, label=''):
        self.lenses.append(Lens(position, focal_length, label))
        self.lenses.sort(key=lambda x: x.position)
        return self
    
    def get_q_at(self, z):
        """Get complex beam parameter at position z"""
        if self.w0 is None:
            raise ValueError("Must seed waist first")
        
        # Start with q at the initial waist
        zR = np.pi * self.w0**2 / self.wavelength
        q = 1j * zR  # At waist, q = i*zR
        current_z = self.z0
        
        # Propagate through optical system
        for lens in self.lenses:
            if lens.position > z:
                break
            
            # Free space propagation to lens
            d = lens.position - current_z
            if d != 0:
                q = q + d
            
            # Thin lens transformation: 1/q' = 1/q - 1/f
            # Equivalently: q' = q / (1 - q/f)
            q = q / (1 - q/lens.focal_length)
            current_z = lens.position
        
        # Final free space propagation
        d = z - current_z
        if d != 0:
            q = q + d
        
        return q
    
    def beam_width_at(self, z):
        """Get beam radius at position z"""
        q = self.get_q_at(z)
        # Beam width from q: w = sqrt(lambda/(pi*Im(1/q)))
        q_inv = 1.0 / q
        w = np.sqrt(-self.wavelength / (np.pi * q_inv.imag))
        return w
    
    def gouy_phase_at(self, z, wrap=False):
        """Get Gouy phase at position z"""
        q = self.get_q_at(z)
        # Gouy phase = arctan(Re(q)/Im(q))
        phase = np.arctan2(q.real, q.imag) * 180 / np.pi
        if wrap:
            phase = np.mod(phase + 180, 360) - 180
        return phase
    
    def set_target_waist(self, w0_target, z0_target):
        self.target_w0 = w0_target
        self.target_z0 = z0_target
        return self
    
    def get_overlap(self):
        """Calculate mode overlap with target"""
        if self.target_w0 is None:
            return None
        
        z = self.target_z0
        
        # Get actual beam parameters at target
        q_actual = self.get_q_at(z)
        w_actual = self.beam_width_at(z)
        
        # Calculate radius of curvature
        q_inv = 1.0 / q_actual
        if np.abs(q_inv.real) < 1e-12:
            R_actual = np.inf
        else:
            R_actual = 1.0 / q_inv.real
        
        # Target beam parameters
        w_target = self.target_w0
        R_target = np.inf  # Target is at its waist
        
        # Mode overlap formula
        if R_actual == np.inf and R_target == np.inf:
            phase_term = 0
        elif R_actual == np.inf:
            phase_term = (np.pi * w_actual * w_target / self.wavelength / R_target)**2
        elif R_target == np.inf:
            phase_term = (np.pi * w_actual * w_target / self.wavelength / R_actual)**2
        else:
            phase_term = (np.pi * w_actual * w_target / self.wavelength * 
                         (1/R_actual - 1/R_target))**2
        
        size_term = (w_actual/w_target + w_target/w_actual)**2
        overlap = 4 / (size_term + phase_term)
        
        return overlap
    
    def optimize_lenses(self, lens_labels, bounds_dict):
        if self.target_w0 is None:
            raise ValueError("Must set target waist first")
        
        lens_indices = []
        for label in lens_labels:
            for i, lens in enumerate(self.lenses):
                if lens.label == label:
                    lens_indices.append(i)
                    break
        
        if not lens_indices:
            return self
        
        x0 = [self.lenses[i].position for i in lens_indices]
        bounds = [bounds_dict[self.lenses[i].label] for i in lens_indices]
        
        def objective(positions):
            for idx, pos in zip(lens_indices, positions):
                self.lenses[idx].position = pos
            self.lenses.sort(key=lambda x: x.position)
            overlap = self.get_overlap()
            return -overlap
        
        result = minimize(objective, x0, bounds=bounds, method='L-BFGS-B')
        
        for idx, pos in zip(lens_indices, result.x):
            self.lenses[idx].position = pos
        self.lenses.sort(key=lambda x: x.position)
        
        return self
    
    def copy(self):
        return deepcopy(self)

## Interactive GUI Application

In [10]:
class BeamMatchingGUI:
    def __init__(self):
        # Data storage
        self.z_data_x = None
        self.w_data_x = None
        self.z_data_y = None
        self.w_data_y = None
        self.beam_x = None
        self.beam_y = None
        self.beam_opt = None
        self.wavelength = 1064e-9
        
        # Create GUI components
        self.create_widgets()
        
    def create_widgets(self):
        # Header
        self.header = widgets.HTML(
            value="<h2>Gaussian Beam Mode Matching Tool</h2>",
            layout=widgets.Layout(margin='0 0 20px 0')
        )
        
        # Tab structure
        self.tab = widgets.Tab()
        
        # Tab 1: Beam Profile Input
        self.create_beam_input_tab()
        
        # Tab 2: Lens Configuration
        self.create_lens_config_tab()
        
        # Tab 3: Results
        self.create_results_tab()
        
        self.tab.children = [self.beam_tab, self.lens_tab, self.results_tab]
        self.tab.set_title(0, 'Beam Profile')
        self.tab.set_title(1, 'Lens Setup')
        self.tab.set_title(2, 'Results')
        
        # Output area
        self.output = widgets.Output()
        
    def create_beam_input_tab(self):
        """Create the beam profile input interface"""
        # Wavelength input
        self.wavelength_input = widgets.FloatText(
            value=1064,
            description='Wavelength (nm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='300px')
        )
        
        # Data input method selector
        self.input_method = widgets.RadioButtons(
            options=['Manual Entry', 'Load NPRO Example'],
            value='Load NPRO Example',
            description='Input Method:',
            style={'description_width': '150px'}
        )
        
        # Manual entry fields
        self.z_positions_x = widgets.Textarea(
            value='',
            placeholder='Enter z positions (mm), comma separated\nExample: 281, 291, 301, 311, ...',
            description='X: Z positions (mm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='600px', height='80px')
        )
        
        self.w_measurements_x = widgets.Textarea(
            value='',
            placeholder='Enter beam widths (μm), comma separated\nExample: 1628.7, 1663.4, 1701.8, ...',
            description='X: Beam widths (μm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='600px', height='80px')
        )
        
        self.z_positions_y = widgets.Textarea(
            value='',
            placeholder='Enter z positions (mm), comma separated',
            description='Y: Z positions (mm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='600px', height='80px')
        )
        
        self.w_measurements_y = widgets.Textarea(
            value='',
            placeholder='Enter beam widths (μm), comma separated',
            description='Y: Beam widths (μm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='600px', height='80px')
        )
        
        self.manual_entry_box = widgets.VBox([
            widgets.HTML("<h4>X-axis measurements:</h4>"),
            self.z_positions_x,
            self.w_measurements_x,
            widgets.HTML("<h4>Y-axis measurements:</h4>"),
            self.z_positions_y,
            self.w_measurements_y
        ])
        
        # Button to fit beam
        self.fit_button = widgets.Button(
            description='🎯 Fit Beam Profile',
            button_style='success',
            layout=widgets.Layout(width='200px', height='40px')
        )
        self.fit_button.on_click(self.fit_beam_profile)
        
        # Status display
        self.beam_status = widgets.HTML(value="<i>No beam fitted yet</i>")
        
        self.beam_tab = widgets.VBox([
            self.wavelength_input,
            widgets.HTML("<hr>"),
            self.input_method,
            self.manual_entry_box,
            widgets.HTML("<hr>"),
            self.fit_button,
            self.beam_status
        ], layout=widgets.Layout(padding='20px'))
        
        # Update visibility based on input method
        self.input_method.observe(self.update_input_method, 'value')
        self.update_input_method(None)
        
    def update_input_method(self, change):
        if self.input_method.value == 'Load NPRO Example':
            self.manual_entry_box.layout.display = 'none'
        else:
            self.manual_entry_box.layout.display = 'block'
    
    def create_lens_config_tab(self):
        """Create lens configuration interface"""
        # Lens list container
        self.lens_widgets = []
        self.lens_container = widgets.VBox([])
        
        # Add lens button
        self.add_lens_button = widgets.Button(
            description='➕ Add Lens',
            button_style='primary',
            layout=widgets.Layout(width='150px')
        )
        self.add_lens_button.on_click(self.add_lens_widget)
        
        # Target waist settings
        self.target_w0 = widgets.FloatText(
            value=3.3,
            description='Target waist (μm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='300px')
        )
        
        self.target_z0 = widgets.FloatText(
            value=403,
            description='Target position (mm):',
            style={'description_width': '150px'},
            layout=widgets.Layout(width='300px')
        )
        
        # Optimize button
        self.optimize_button = widgets.Button(
            description='🚀 Optimize Lens Positions',
            button_style='success',
            layout=widgets.Layout(width='250px', height='40px')
        )
        self.optimize_button.on_click(self.optimize_system)
        
        # Calculate button (without optimization)
        self.calculate_button = widgets.Button(
            description='📊 Calculate (No Optimization)',
            button_style='info',
            layout=widgets.Layout(width='250px', height='40px')
        )
        self.calculate_button.on_click(self.calculate_system)
        
        self.lens_status = widgets.HTML(value="<i>Add lenses and configure target</i>")
        
        self.lens_tab = widgets.VBox([
            widgets.HTML("<h3>Optical Elements</h3>"),
            self.add_lens_button,
            self.lens_container,
            widgets.HTML("<hr><h3>Target Waist</h3>"),
            self.target_w0,
            self.target_z0,
            widgets.HTML("<hr>"),
            widgets.HBox([self.calculate_button, self.optimize_button]),
            self.lens_status
        ], layout=widgets.Layout(padding='20px'))
        
        # Add default lenses
        self.add_lens_widget(None)
        self.add_lens_widget(None)
    
    def add_lens_widget(self, b):
        """Add a new lens configuration widget"""
        n = len(self.lens_widgets) + 1
        
        label = widgets.Text(
            value=f'L{n}',
            description='Label:',
            layout=widgets.Layout(width='150px')
        )
        
        focal_length = widgets.FloatText(
            value=150 if n == 1 else 5,
            description='Focal length (mm):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='250px')
        )
        
        position = widgets.FloatText(
            value=140 if n == 1 else 400,
            description='Position (mm):',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='250px')
        )
        
        optimize_check = widgets.Checkbox(
            value=True,
            description='Optimize',
            layout=widgets.Layout(width='100px')
        )
        
        min_pos = widgets.FloatText(
            value=102 if n == 1 else 200,
            description='Min (mm):',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='170px')
        )
        
        max_pos = widgets.FloatText(
            value=178 if n == 1 else 700,
            description='Max (mm):',
            style={'description_width': '70px'},
            layout=widgets.Layout(width='170px')
        )
        
        remove_button = widgets.Button(
            description='❌',
            button_style='danger',
            layout=widgets.Layout(width='50px')
        )
        
        lens_box = widgets.VBox([
            widgets.HBox([label, focal_length, position, optimize_check, remove_button]),
            widgets.HBox([widgets.HTML("<span style='margin-left:155px'>Optimization bounds:</span>"), 
                         min_pos, max_pos])
        ], layout=widgets.Layout(border='1px solid #ccc', padding='10px', margin='5px'))
        
        lens_data = {
            'box': lens_box,
            'label': label,
            'focal_length': focal_length,
            'position': position,
            'optimize': optimize_check,
            'min_pos': min_pos,
            'max_pos': max_pos,
            'remove_button': remove_button
        }
        
        remove_button.on_click(lambda b: self.remove_lens_widget(lens_data))
        
        self.lens_widgets.append(lens_data)
        self.update_lens_container()
    
    def remove_lens_widget(self, lens_data):
        """Remove a lens widget"""
        self.lens_widgets.remove(lens_data)
        self.update_lens_container()
    
    def update_lens_container(self):
        """Update the lens container display"""
        self.lens_container.children = [ld['box'] for ld in self.lens_widgets]
    
    def create_results_tab(self):
        """Create results display"""
        self.results_output = widgets.Output()
        
        self.results_tab = widgets.VBox([
            widgets.HTML("<h3>Analysis Results</h3>"),
            self.results_output
        ], layout=widgets.Layout(padding='20px'))
    
    def fit_beam_profile(self, b):
        """Fit the beam profile from measurements"""
        with self.output:
            clear_output(wait=True)
            
            # Update wavelength
            self.wavelength = self.wavelength_input.value * 1e-9
            
            if self.input_method.value == 'Load NPRO Example':
                # Load example data
                z_offset = 0.383 + -0.102
                self.z_data_x = np.arange(0, 110, 10) * 1e-3 + z_offset
                self.w_data_x = np.array([1628.7, 1663.4, 1701.8, 1739.2, 1777.3, 1822.1, 
                                          1855.3, 1904.1, 1926.3, 1958.7, 2008.8]) * 1e-6 / 2
                self.z_data_y = self.z_data_x.copy()
                self.w_data_y = np.array([1739.5, 1775.0, 1805.1, 1848.5, 1878.2, 1924.4, 
                                          1967.9, 2007.8, 2024.7, 2064.9, 2097.5]) * 1e-6 / 2
            else:
                # Parse manual input
                try:
                    z_x_str = [x.strip() for x in self.z_positions_x.value.split(',')]
                    w_x_str = [x.strip() for x in self.w_measurements_x.value.split(',')]
                    z_y_str = [x.strip() for x in self.z_positions_y.value.split(',')]
                    w_y_str = [x.strip() for x in self.w_measurements_y.value.split(',')]
                    
                    self.z_data_x = np.array([float(x) for x in z_x_str]) * 1e-3  # mm to m
                    self.w_data_x = np.array([float(x) for x in w_x_str]) * 1e-6  # μm to m
                    self.z_data_y = np.array([float(x) for x in z_y_str]) * 1e-3
                    self.w_data_y = np.array([float(x) for x in w_y_str]) * 1e-6
                except:
                    print("❌ Error parsing input data. Check format.")
                    return
            
            # Fit beams
            print("Fitting beam profiles...")
            w0_x, z0_x, r2_x = fit_gaussian_beam(self.z_data_x, self.w_data_x, self.wavelength)
            w0_y, z0_y, r2_y = fit_gaussian_beam(self.z_data_y, self.w_data_y, self.wavelength)
            
            # Create beam objects
            self.beam_x = BeamPath(wavelength=self.wavelength)
            self.beam_x.seed_waist(w0_x, z0_x)
            
            self.beam_y = BeamPath(wavelength=self.wavelength)
            self.beam_y.seed_waist(w0_y, z0_y)
            
            # Plot
            fig, ax = plt.subplots(figsize=(14, 6))
            
            ax.plot(self.z_data_x, self.w_data_x, 'bo', markersize=8, label='X measurements', zorder=5)
            ax.plot(self.z_data_y, self.w_data_y, 'ro', markersize=8, label='Y measurements', zorder=5)
            
            # Plot range should include the waists
            z_plot_min = min(z0_x, z0_y, self.z_data_x.min()) - 0.2
            z_plot_max = max(self.z_data_x.max(), self.z_data_y.max()) + 0.1
            z_plot = np.linspace(z_plot_min, z_plot_max, 1000)
            w_x_fit = beam_width(z_plot, w0_x, z0_x, self.wavelength)
            w_y_fit = beam_width(z_plot, w0_y, z0_y, self.wavelength)
            
            ax.plot(z_plot, w_x_fit, 'b-', linewidth=2, label='X fit')
            ax.plot(z_plot, -w_x_fit, 'b-', linewidth=2)
            ax.plot(z_plot, w_y_fit, 'r-', linewidth=2, label='Y fit')
            ax.plot(z_plot, -w_y_fit, 'r-', linewidth=2)
            
            ax.axvline(z0_x, color='b', linestyle='--', alpha=0.5, linewidth=1.5)
            ax.axvline(z0_y, color='r', linestyle='--', alpha=0.5, linewidth=1.5)
            
            ax.set_xlabel('Distance [m]', fontsize=12)
            ax.set_ylabel('Beam Radius [m]', fontsize=12)
            ax.set_title(f'Fitted Beam: X waist = {w0_x*1e6:.1f} µm at {z0_x:.3f} m (R²={r2_x:.4f}), '
                        f'Y waist = {w0_y*1e6:.1f} µm at {z0_y:.3f} m (R²={r2_y:.4f})', fontsize=12)
            ax.grid(True, alpha=0.3)
            ax.legend(fontsize=11)
            plt.tight_layout()
            plt.show()
            
            self.beam_status.value = f"<b>✓ Beam fitted successfully!</b><br>X: {w0_x*1e6:.1f} µm @ {z0_x*1000:.1f} mm<br>Y: {w0_y*1e6:.1f} µm @ {z0_y*1000:.1f} mm"
    
    def calculate_system(self, b):
        """Calculate system without optimization"""
        self._run_analysis(optimize=False)
    
    def optimize_system(self, b):
        """Optimize lens positions"""
        self._run_analysis(optimize=True)
    
    def _run_analysis(self, optimize=False):
        """Run the analysis with or without optimization"""
        if self.beam_x is None or self.beam_y is None:
            with self.output:
                clear_output(wait=True)
                print("❌ Please fit beam profile first (Tab 1)")
            return
        
        with self.results_output:
            clear_output(wait=True)
            
            # Create beam copies
            beam_x = self.beam_x.copy()
            beam_y = self.beam_y.copy()
            
            # Add lenses
            for lens_data in self.lens_widgets:
                f = lens_data['focal_length'].value * 1e-3  # mm to m
                z = lens_data['position'].value * 1e-3  # mm to m
                label = lens_data['label'].value
                
                beam_x.add_lens(f, z, label)
                beam_y.add_lens(f, z, label)
            
            # Set target
            target_w = self.target_w0.value * 1e-6  # μm to m
            target_z = self.target_z0.value * 1e-3  # mm to m
            
            beam_x.set_target_waist(target_w, target_z)
            beam_y.set_target_waist(target_w, target_z)
            
            if optimize:
                print("🔧 Optimizing lens positions...\n")
                
                # Create average beam for optimization
                w0_avg = (self.beam_x.w0 + self.beam_y.w0) / 2
                z0_avg = (self.beam_x.z0 + self.beam_y.z0) / 2
                beam_opt = BeamPath(wavelength=self.wavelength)
                beam_opt.seed_waist(w0_avg, z0_avg)
                
                for lens_data in self.lens_widgets:
                    f = lens_data['focal_length'].value * 1e-3
                    z = lens_data['position'].value * 1e-3
                    label = lens_data['label'].value
                    beam_opt.add_lens(f, z, label)
                
                beam_opt.set_target_waist(target_w, target_z)
                
                # Build optimization settings
                opt_labels = []
                bounds_dict = {}
                
                for lens_data in self.lens_widgets:
                    if lens_data['optimize'].value:
                        label = lens_data['label'].value
                        opt_labels.append(label)
                        bounds_dict[label] = (
                            lens_data['min_pos'].value * 1e-3,
                            lens_data['max_pos'].value * 1e-3
                        )
                
                if opt_labels:
                    beam_opt.optimize_lenses(opt_labels, bounds_dict)
                    
                    # Apply optimized positions
                    for i, lens in enumerate(beam_opt.lenses):
                        beam_x.lenses[i].position = lens.position
                        beam_y.lenses[i].position = lens.position
                        # Update GUI
                        self.lens_widgets[i]['position'].value = lens.position * 1e3  # m to mm
                    
                    beam_x.lenses.sort(key=lambda x: x.position)
                    beam_y.lenses.sort(key=lambda x: x.position)
                    
                    print("✓ Optimization complete!\n")
            
            # Calculate overlaps
            overlap_x = beam_x.get_overlap()
            overlap_y = beam_y.get_overlap()
            mode_match = np.sqrt(overlap_x * overlap_y)
            
            # Display results
            print("=" * 70)
            print("RESULTS")
            print("=" * 70)
            print(f"\nMode Matching Performance:")
            print(f"  X-axis overlap:    {overlap_x:.4f} ({overlap_x*100:.2f}%)")
            print(f"  Y-axis overlap:    {overlap_y:.4f} ({overlap_y*100:.2f}%)")
            print(f"  Combined overlap:  {mode_match:.4f} ({mode_match*100:.2f}%)")
            
            print(f"\nLens Positions:")
            for lens in beam_x.lenses:
                print(f"  {lens.label}: z = {lens.position*1000:.2f} mm, f = {lens.focal_length*1000:.1f} mm")
            
            print(f"\nBeam at Target (z = {target_z*1000:.1f} mm):")
            wx = beam_x.beam_width_at(target_z)
            wy = beam_y.beam_width_at(target_z)
            print(f"  X-beam width: {wx*1e6:.2f} µm")
            print(f"  Y-beam width: {wy*1e6:.2f} µm")
            print(f"  Target width: {target_w*1e6:.2f} µm")
            print(f"  Ellipticity:  {max(wx,wy)/min(wx,wy):.3f}")
            print("\n" + "="*70)
            
            # Plot results
            fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
            
            z_min = min(self.beam_x.z0, self.beam_y.z0) - 0.1
            z_max = target_z + 0.1
            z_range = np.linspace(z_min, z_max, 2000)
            
            # Beam width plot
            w_y = np.array([beam_y.beam_width_at(z) for z in z_range])
            w_x = np.array([beam_x.beam_width_at(z) for z in z_range])
            
            ax1.plot(z_range, w_y, 'b-', linewidth=2, label='vertical')
            ax1.plot(z_range, -w_y, 'b-', linewidth=2)
            ax1.plot(z_range, w_x, 'r-', linewidth=2, label='horizontal')
            ax1.plot(z_range, -w_x, 'r-', linewidth=2)
            
            # Mark lenses
            for lens in beam_x.lenses:
                ax1.axvline(lens.position, color='gray', linestyle=':', alpha=0.5, linewidth=2)
                ax1.text(lens.position, ax1.get_ylim()[1]*0.92, lens.label, 
                        ha='center', fontsize=11, weight='bold')
            
            # Mark target
            ax1.axvline(target_z, color='green', linestyle='--', alpha=0.5, linewidth=2, label='target')
            
            ax1.set_ylabel('Beam width (m)', fontsize=12)
            ax1.set_title(f'Mode Matching = {mode_match:.4f}', fontsize=14, weight='bold')
            ax1.grid(True, alpha=0.3)
            ax1.legend(fontsize=11)
            
            # Gouy phase plot
            phase_y = np.array([beam_y.gouy_phase_at(z, wrap=True) for z in z_range])
            phase_x = np.array([beam_x.gouy_phase_at(z, wrap=True) for z in z_range])
            
            ax2.plot(z_range, phase_y, 'b-', linewidth=2, label='vertical')
            ax2.plot(z_range, phase_x, 'r-', linewidth=2, label='horizontal')
            
            for lens in beam_x.lenses:
                ax2.axvline(lens.position, color='gray', linestyle=':', alpha=0.5, linewidth=2)
            
            ax2.set_ylabel('Gouy Phase (degrees)', fontsize=12)
            ax2.set_xlabel('Axial distance (m)', fontsize=12)
            ax2.grid(True, alpha=0.3)
            ax2.legend(fontsize=11)
            
            plt.tight_layout()
            plt.show()
            
            status_text = "🎯 Optimization" if optimize else "📊 Calculation"
            self.lens_status.value = f"<b>{status_text} complete!</b><br>Mode matching: {mode_match:.4f}"
    
    def display(self):
        """Display the GUI"""
        display(self.header)
        display(self.tab)
        display(self.output)

print("✓ GUI class defined")

✓ GUI class defined


## Launch the Interactive Tool

In [11]:
# Create and display the GUI
gui = BeamMatchingGUI()
gui.display()

HTML(value='<h2>Gaussian Beam Mode Matching Tool</h2>', layout=Layout(margin='0 0 20px 0'))

Output()